Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

T.b.d.


In [690]:
import logging
import random
import numpy as np
import functools
from typing import Callable
from itertools import accumulate
from copy import deepcopy
from operator import xor
from collections import namedtuple

random.seed(42)

In [691]:
Nimply = namedtuple("Nimply", "row, num_objects")
Move = namedtuple("Move", "row num_objects fitness")

## NIM Game 

In [692]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k
    
    def __str__(self):
        return f"{self._rows}"

    def nimming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        assert num_objects > 0, f"You have to pick at least one"
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.debug("Yeuch")
    
    def nimming2(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        assert num_objects > 0, f"You have to pick at least one"
        self._rows[row] -= num_objects

    @property
    def rows(self):
        return self._rows

    @property
    def k(self) -> int:
        return self._k

In [693]:
def nim_sum(rows: list) -> int:
    # List XOR
    # Using reduce() + lambda + "^" operator
    res = functools.reduce(lambda x, y: x ^ y, rows)
    return res


In [694]:
def tournament(population, tournament_size=2):
    return min(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

def mutation(p: Move, nim: Nim):
    if nim.k is None:
        elements = random.randrange(1, nim.rows[p.row] + 1)
        temp_rows = nim.rows.copy()
        temp_rows[p.row] -=elements 
        offspring = Move(p.row, elements , nim_sum(temp_rows))
    else:
        elements = min(nim.k, random.randrange(1, nim.rows[p.row] + 1))
        temp_rows = nim.rows.copy()
        temp_rows[p.row] -=elements 
        offspring = Move(p.row, elements , nim_sum(temp_rows))

    return offspring

def cross_over(p1: Move, p2: Move, nim: Nim):

    n_random = random.randint(0, 1)
    
    if n_random == 0:

        temp_rows = nim.rows.copy()
        temp_rows[p1.row] -= p2.num_objects

        if temp_rows[p1.row] < 0:
            return None

        offspring = Move(p1.row, p2.num_objects , nim_sum(temp_rows))
    else:
        temp_rows = nim.rows.copy()
        temp_rows[p2.row] -= p1.num_objects

        if temp_rows[p2.row] < 0:
            return None

        offspring = Move(p2.row, p1.num_objects , nim_sum(temp_rows))
    
    if offspring.num_objects > nim.k:
        return None

    return offspring


## Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)

### Creating the NIM Table 

In [695]:
N_ROWS = 5
GAMEOVER = [0 for _ in range(N_ROWS)]
POPULATION_SIZE = 10
OFFSPRING_SIZE = 10
N_GENERATIONS = 20
K = 5



In [696]:
def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state.rows)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming2(m)
        brute_force.append((m, nim_sum(tmp.rows)))
    cooked["brute_force"] = brute_force

    return cooked

### Player:

#### Strategies:

* Task 3.1 (_Expert System_) : _Best Possibile Strategy_ Implemented in `best_strategy` and nominated as `best` (see below). To understand the algorithm of the winning strategy, look at [*Nim*](https://en.wikipedia.org/wiki/Nim)!
* 


In [697]:
class Player:
    def __init__(self, strategy = 'best') -> None:
        # Two parts for the best strategy:
        # 0 -> before all rows have one element
        # 1 -> after all rows have one element
        self._best_strategy = 0

        assert strategy in ['best', 'best_prof', 'pure_random', 'ga', 'evolvable', "evolvable_prof"], f"Strategy non-available"
        self._strategy = strategy

    def moves(self, Nim, alpha = 0.5, beta = 0.5):
        if self._strategy == 'best':
            return self.best_strategy(Nim)
        elif self._strategy == 'best_prof':
            return self.best_strategy_by_prof(Nim)
        elif self._strategy == 'pure_random':
            return self.pure_random(Nim)
        elif self._strategy == 'ga':
            return self.evolvable_based_on_GA(Nim)
        elif self._strategy == 'evolvable':
            return self.evolvable_based_on_fixed_rules(Nim, cook_status(Nim), alpha, beta)
        elif self._strategy == 'evolvable_prof':
            return self.evolvable_by_prof(Nim, alpha)
        else: 
            assert f"Can't use a strategy"
        return


    def pure_random(self, Nim):

        # The opponent choose randomly a non-empty row 
        nonzeroind = np.nonzero(Nim.rows)[0]
        random_row = random.choice(nonzeroind)

        # The opponen choose to remove a random number of elements
        if Nim._k == None:
            random_elements = random.randint(1,Nim.rows[random_row])
        else:
            random_elements = random.randint(1,min(Nim._k,Nim.rows[random_row]))

        return Nimply(random_row, random_elements)

        
    def best_strategy(self, Nim):

        # If all the elements are equal or less then k, we can play the 'normal' nim game
        if Nim._k != None and all(v <= Nim._k for v in Nim.rows):
            temp_k = None
        else:
            temp_k = Nim._k

        if temp_k != None:

            # Try brute force:
            for ind, row in enumerate(Nim.rows):

                for el in range(1, min(row + 1, Nim._k + 1)):
                    # Reset temp_rows
                    temp_rows = Nim.rows.copy()
                    
                    # See if nim_sum == 0
                    temp_rows[ind] -= el
                    if nim_sum(temp_rows) == 0:
                        # Update table
                        # Nim.nimming(ind, el)
                        return Nimply(ind, el)
            
            equal_grater_than_k_ind = [i for i,v in enumerate(Nim.rows) if v >= Nim._k + 1]
            
            random_row = random.choice(equal_grater_than_k_ind)
            elements = Nim.rows[random_row]%(Nim._k+1) 
            
            if elements == 0:
                elements = 1

            return Nimply(random_row, elements)

        # If there is only one element greater to one, the agent picks a number of object to make
        # all the rows of the table equal to 1.
        # He can choose to remove all the objects or all the objects but one from the rows with n>1
        if sum(x >= 2 for x in Nim.rows) == 1:
            # Row with more than one element
            equal_grater_than_two_ind = [i for i,v in enumerate(Nim.rows) if v >= 2][0]

            # Change of strategy
            self._best_strategy = 1

            
            # To win, the remaing number of objects has to be even 
            if (sum(x for x in Nim.rows) - Nim.rows[equal_grater_than_two_ind]) % 2 == 0 :
        
                return Nimply(equal_grater_than_two_ind, Nim.rows[equal_grater_than_two_ind])
                
            else:

                return Nimply(equal_grater_than_two_ind, Nim.rows[equal_grater_than_two_ind]-1)        
        
        # Strategy before all rows have one element
        if self._best_strategy == 0:    
        
            res = nim_sum(Nim.rows)

            for ind, row in enumerate(Nim.rows):

                if row == 0:
                    continue

                if row ^ res < row:
                   
                    elements = row - (row ^ res)

                    return Nimply(ind, elements)
        
        # Strategy after all rows have one element
        else:

            nonzeroind = [i for i, e in enumerate(Nim.rows) if e != 0]
            random_row = random.choice(nonzeroind)

            return Nimply(random_row, 1) 
                 
        # Default move -> Random
        nonzeroind = [i for i, e in enumerate(Nim.rows) if e != 0]
        random_row = random.choice(nonzeroind)

        if Nim._k == None:
            random_elements = random.randrange(1,Nim.rows[random_row] + 1)
        else:
            random_elements = random.randrange(1,min(Nim._k,Nim.rows[random_row])+1)

          
        return Nimply(random_row, random_elements) 

    def best_strategy_by_prof(self, state: Nim):
        data = cook_status(state)
        # state.nimming2(next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0])
        move  = next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]
        return Nimply(move[0], move[1])

    def evolvable_by_prof(self, state: Nim, p = 0.5):
        data = cook_status(state)

        if random.random() < p:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply
    
    def evolvable_based_on_fixed_rules(self, state: Nim, cook_status: dict, alpha: float = 0.5, beta: float = 0.5):
        initial_numbers = sum([i*2 + 1 for i in range(N_ROWS)])
        actual_numbers = sum(state.rows)

        # Early game strategy
        if actual_numbers > alpha * initial_numbers:
            
            if cook_status['active_rows_number'] >= beta*N_ROWS:

                row = cook_status["longest_row"]
                if state.k is not None:
                    elements = min(state.k, state.rows[row])

                else:
                    elements = state.rows[row]

                # state.nimming(row, elements)
                return Nimply(row, elements)

        row = cook_status["longest_row"]

        if cook_status["active_rows_number"]%2 == 0 and state.rows[row]!=1 :
            #Leave at least one element
            elements = min( (state.k - 1), state.rows[row] - 1)

        else:
            #Try to remove the maximum number of elements
            elements = min( state.k, state.rows[row])
            
        # state.nimming(row, elements)
        return Nimply(row, elements)
        
    def evolvable_based_on_GA(self, state: Nim):
        
        # Population = possible moves
        population = []
        for _ in range(POPULATION_SIZE):
            # temp_rows needed to evaluate nim_sum = fitness (low nim_sum is better!)
            temp_rows = state.rows.copy()

            # Choosing a random_row
            nonzeroind = [i for i, e in enumerate(state.rows) if e != 0]
            random_row = random.choice(nonzeroind)

            #Choosing a random_move
            if state.k is None:
                random_elements = random.randrange(1, state.rows[random_row] + 1)

            else:
                random_elements =  min(random.randrange(1, state.rows[random_row] + 1), state.k)

            temp_rows[random_row] -= random_elements
            fitness = nim_sum(temp_rows)
            pop = Move(random_row, random_elements, fitness)
            population.append(pop)
        

        for g in range(N_GENERATIONS):
            offspring = list()
            for i in range(OFFSPRING_SIZE):

                if random.random() < 0.5:
                    # Selection of parents
                    p = tournament(population.copy())

                    # Offspring generation
                    o = mutation(p, state)       

                else:
                    
                    p1 = tournament(population)
                    p2 = tournament(population)
                    o = cross_over(p1, p2, state)

                # Check if cross-over returned a valid solution.
                # In this code, only valid solutions has been considered.
                # Possible Improvement: Acceptance with penalties of non-valid solutions
                if o == None:
                    continue

                offspring.append(o)
            
            # Adding new Offspings generated to Population list
            population+=offspring
            
            # Sorting the Population, according to their fitness and selecting the firsts n_elements = POPULATION_SIZE
            population = sorted(population, key=lambda i: i.fitness, reverse=False)[:POPULATION_SIZE]
            logging.debug(f"actual best {population[0]}")
        
        
        # state.nimming(population[0].row, population[0].num_objects)
        return Nimply(population[0].row, population[0].num_objects)

### Single Match

In [698]:
def single_match(agent_strategy = 'best', opponent_strategy = 'pure_random'):
    
    agent = Player(agent_strategy)
    opponent = Player(opponent_strategy)

    Table = Nim(N_ROWS, K)

    # 0 -> Agent's turn
    # 1 -> Opponent's turn
    turn = 1

    # Game
    while Table._rows != GAMEOVER:
        
        if turn == 0:
            agent.moves(Table)
        else:
            opponent.moves(Table)
        
        turn = 1 - turn
        
    # Game Over

    if turn == 1:
        print(f"Agent WON the match")
    else:
        print(f"Opponent WON the match")
    
    return



### Multiple Games

In [699]:
NUM_MATCHES = 5


def evaluate(agent_strategy = 'best', opponent_strategy = 'pure_random', parameter_dict: dict = {"alpha": None, "beta": None}) -> float:
    
    
    won = 0
    start = 0
    for m in range(NUM_MATCHES):
        agent = Player(agent_strategy)
        opponent = Player(opponent_strategy)

        
        nim = Nim(N_ROWS, K)
        
        
        # 0 -> Agent's turn
        # 1 -> Opponent's turn
        turn = start

        # the first move is equally distributed within matches
        start = 1 - start 
        
        # turn = random.randint(0,1)

        logging.debug(f"\n\n\n--------NEW GAME---------")
        # Game
        while nim._rows != GAMEOVER:
            if turn == 0:
                logging.debug(f" Actual turn: Agent")
            else:
                logging.debug(f" Actual turn: Opponent")

            logging.debug(f" \tTable before move: {nim} and Nim_sum: {nim_sum(nim._rows)}")
            
            if turn == 0:
                if agent_strategy == 'evolvable':
                    ply = agent.moves(nim, parameter_dict['alpha'], parameter_dict['beta'] )
                    assert parameter_dict['alpha'] is not None, f"Please choose a value for alfa"
                    assert parameter_dict['beta'] is not None, f"Please choose a value for beta"

                elif agent_strategy == 'evolvable_by_prof':
                    assert parameter_dict['alpha'] is not None, f"Please choose a value for alfa"
                    ply = agent.moves(nim, parameter_dict['alpha'])
                else:
                    ply = agent.moves(nim)
                
                logging.debug(f" \tAgent:   <Row: {ply.row}- Elements: {ply.num_objects}>")
                
            else:
                if opponent_strategy == 'evolvable':
                    ply = opponent.moves(nim, parameter_dict['alpha_opp'], parameter_dict['beta_opp'] )
                else:
                    ply = opponent.moves(nim)
                
                logging.debug(f" \tOpponent:   <Row: {ply.row}- Elements: {ply.num_objects}>")


            nim.nimming2(ply)
            logging.debug(f" \tTable after move: {nim} and Nim_sum: {nim_sum(nim._rows)}\n")

            
            turn = 1 - turn
        
        logging.debug(f"--------GAME OVER---------")
        # Game Over
        if turn == 1:
            won +=1
        else:
            print(f"Game Lost by the agent is the n°{m}")
            
        
    return won / NUM_MATCHES


In [700]:
logging.getLogger().setLevel(logging.DEBUG)
parameter_dict= {}
parameter_dict["alpha"] = 0.5
parameter_dict["beta"] = 0.5
parameter_dict["alpha_opp"] = 0.99
parameter_dict["beta_opp"] = 0.1
print(f"Agent Won: {evaluate()*100}% of the games")
print(f"Agent Won: {evaluate(agent_strategy='best_prof')*100}% of the games")
print(f"Agent Won: {evaluate(agent_strategy='best', opponent_strategy='best_prof')*100}% of the games")
print(f"Agent Won: {evaluate(agent_strategy='ga', opponent_strategy='best_prof')*100}% of the games")
print(f"Agent Won: {evaluate(agent_strategy='evolvable', opponent_strategy='evolvable', parameter_dict = parameter_dict)*100}% of the games")

DEBUG:root:


--------NEW GAME---------
DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 9] and Nim_sum: 9
DEBUG:root: 	Agent:   <Row: 3- Elements: 1>
DEBUG:root: 	Table after move: [1, 3, 5, 6, 9] and Nim_sum: 8

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 5, 6, 9] and Nim_sum: 8
DEBUG:root: 	Opponent:   <Row: 0- Elements: 1>
DEBUG:root: 	Table after move: [0, 3, 5, 6, 9] and Nim_sum: 9

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [0, 3, 5, 6, 9] and Nim_sum: 9
DEBUG:root: 	Agent:   <Row: 3- Elements: 1>
DEBUG:root: 	Table after move: [0, 3, 5, 5, 9] and Nim_sum: 10

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [0, 3, 5, 5, 9] and Nim_sum: 10
DEBUG:root: 	Opponent:   <Row: 2- Elements: 2>
DEBUG:root: 	Table after move: [0, 3, 3, 5, 9] and Nim_sum: 12

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [0, 3, 3, 5, 9] and Nim_sum: 12
DEBUG:root: 	Agent:   <Row: 4- Elements: 4>
DE

Agent Won: 100.0% of the games


DEBUG:root:


--------NEW GAME---------
DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 9] and Nim_sum: 9
DEBUG:root: 	Opponent:   <Row: 2- Elements: 1>
DEBUG:root: 	Table after move: [1, 3, 4, 7, 9] and Nim_sum: 8

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 4, 7, 9] and Nim_sum: 8
DEBUG:root: 	Agent:   <Row: 2- Elements: 3>
DEBUG:root: 	Table after move: [1, 3, 1, 7, 9] and Nim_sum: 13

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 1, 7, 9] and Nim_sum: 13
DEBUG:root: 	Opponent:   <Row: 4- Elements: 3>
DEBUG:root: 	Table after move: [1, 3, 1, 7, 6] and Nim_sum: 2

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 1, 7, 6] and Nim_sum: 2
DEBUG:root: 	Agent:   <Row: 1- Elements: 2>
DEBUG:root: 	Table after move: [1, 1, 1, 7, 6] and Nim_sum: 0

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 1, 1, 7, 6] and Nim_sum: 0
DEBUG:root: 	Opponent:   <Row: 3- Elements: 4

Agent Won: 100.0% of the games


DEBUG:root: 	Table after move: [0, 0, 0, 0, 0] and Nim_sum: 0

DEBUG:root:--------GAME OVER---------
DEBUG:root:


--------NEW GAME---------
DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 9] and Nim_sum: 9
DEBUG:root: 	Opponent:   <Row: 2- Elements: 4>
DEBUG:root: 	Table after move: [1, 3, 1, 7, 9] and Nim_sum: 13

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 1, 7, 9] and Nim_sum: 13
DEBUG:root: 	Agent:   <Row: 4- Elements: 5>
DEBUG:root: 	Table after move: [1, 3, 1, 7, 4] and Nim_sum: 0

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 1, 7, 4] and Nim_sum: 0
DEBUG:root: 	Opponent:   <Row: 3- Elements: 5>
DEBUG:root: 	Table after move: [1, 3, 1, 2, 4] and Nim_sum: 5

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 1, 2, 4] and Nim_sum: 5
DEBUG:root: 	Agent:   <Row: 4- Elements: 3>
DEBUG:root: 	Table after move: [1, 3, 1, 2, 1] and Nim_sum: 0

DEBUG:root: Actual turn: Opponent
DEBUG:r

...

Game Lost by the agent is the n°4
Agent Won: 40.0% of the games


DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [0, 1, 0, 1, 1] and Nim_sum: 1
DEBUG:root: 	Opponent:   <Row: 1- Elements: 1>
DEBUG:root: 	Table after move: [0, 0, 0, 1, 1] and Nim_sum: 0

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [0, 0, 0, 1, 1] and Nim_sum: 0
DEBUG:root: 	Agent:   <Row: 3- Elements: 1>
DEBUG:root: 	Table after move: [0, 0, 0, 0, 1] and Nim_sum: 1

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [0, 0, 0, 0, 1] and Nim_sum: 1
DEBUG:root: 	Opponent:   <Row: 4- Elements: 1>
DEBUG:root: 	Table after move: [0, 0, 0, 0, 0] and Nim_sum: 0

DEBUG:root:--------GAME OVER---------
DEBUG:root:


--------NEW GAME---------
DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 9] and Nim_sum: 9
DEBUG:root: 	Opponent:   <Row: 4- Elements: 5>
DEBUG:root: 	Table after move: [1, 3, 5, 7, 4] and Nim_sum: 4

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 4] and Nim_sum: 4
DEBUG:

Game Lost by the agent is the n°0


DEBUG:root: 	Agent:   <Row: 4- Elements: 1>
DEBUG:root: 	Table after move: [0, 0, 0, 0, 0] and Nim_sum: 0

DEBUG:root:--------GAME OVER---------
DEBUG:root:


--------NEW GAME---------
DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 9] and Nim_sum: 9
DEBUG:root: 	Agent:   <Row: 4- Elements: 5>
DEBUG:root: 	Table after move: [1, 3, 5, 7, 4] and Nim_sum: 4

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 5, 7, 4] and Nim_sum: 4
DEBUG:root: 	Opponent:   <Row: 3- Elements: 5>
DEBUG:root: 	Table after move: [1, 3, 5, 2, 4] and Nim_sum: 1

DEBUG:root: Actual turn: Agent
DEBUG:root: 	Table before move: [1, 3, 5, 2, 4] and Nim_sum: 1
DEBUG:root: 	Agent:   <Row: 2- Elements: 5>
DEBUG:root: 	Table after move: [1, 3, 0, 2, 4] and Nim_sum: 4

DEBUG:root: Actual turn: Opponent
DEBUG:root: 	Table before move: [1, 3, 0, 2, 4] and Nim_sum: 4
DEBUG:root: 	Opponent:   <Row: 4- Elements: 3>
DEBUG:root: 	Table after move: [1, 3, 0, 2, 1] and Nim_sum: 1


KeyboardInterrupt: 